PIL库取得JPG格式的exif信息是OK的，但是无法对HEIC格式操作。
首先，初步筛选删除对象，仅仅通过文件名进行判断。
然后，通过taken time，file size，location等多种方式确认无误。
最后，是否可以通过图像处理的方式，直接确认两个图片就是同一个内容呢？比如都转化为bitmap，进行抽样像素对比？

两个层次进行foreach嵌套，读取文件名相同的文件，效率很低。应该单独获取两个文件list，然后针对list进行文件名判断。

In [1]:
import glob, os
from pathlib import Path, WindowsPath, PurePosixPath
import pickle
from datetime import datetime
from PIL import Image
import exifread

del_dir = "../iPhoneXR_undocumented/**"
nodel_dir = "../wps_photo/**"

def write_list(list_name,filename):
    # store list in binary file so 'wb' mode
    with open(filename, 'wb') as fp:
        pickle.dump(list_name, fp)
        print('Done writing list into a binary file')
        
# Read list to memory
def read_list(filename):
    # for reading also binary mode is important
    with open(filename, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

In [4]:
nodel_file = []
q_file = []

for filename in glob.iglob(pathname= del_dir, recursive=True):
    if os.path.isfile(filename): # filter dirs
        q_file.append(filename.replace('\\', '/'))
for filename in glob.iglob(pathname= nodel_dir, recursive=True):
    if os.path.isfile(filename): # filter dirs
        nodel_file.append(filename.replace('\\', '/'))
                    

In [5]:
   
write_list(nodel_file, 'nodel.lst')
write_list(q_file, 'q_file.lst')

Done writing list into a binary file
Done writing list into a binary file


windows系统的路径格式和linux的不同，通过实验，用str.replace('\\', '/')方式，将\全部转换为/，是最合适的。Path库并不好用。

In [5]:
print(Path(filename2))
print(WindowsPath(filename2))
print(PurePosixPath(filename2))
print(filename2.replace(os.altsep, os.sep))
print(filename2.replace('\\', os.sep))
print(filename2.replace('\\', '/'))

..\iPhoneXR_undocumented\101APPLE\IMG_1836.JPG
..\iPhoneXR_undocumented\101APPLE\IMG_1836.JPG
../iPhoneXR_undocumented\101APPLE\IMG_1836.JPG
..\iPhoneXR_undocumented\101APPLE\IMG_1836.JPG
../iPhoneXR_undocumented\101APPLE\IMG_1836.JPG
../iPhoneXR_undocumented/101APPLE/IMG_1836.JPG


In [6]:
q_file_r = read_list( 'q_file.lst')
nodel_r = read_list( 'nodel.lst')
delfile = []
reserve_file = []
for f in q_file_r:
    obj_file = Path(f).stem
    for f2 in nodel_r:
        delornot = Path(f2).stem
        if obj_file == delornot:
            delfile.append(f)
            reserve_file.append(f2)
            # print(f+' should be del')

write_list(delfile, 'del_file.lst')
write_list(reserve_file, 'reserve_file.lst')

Done writing list into a binary file
Done writing list into a binary file


In [2]:
delfile = read_list( 'del_file.lst')
reserve_file = read_list( 'reserve_file.lst')

下面对list中的影像文件进行拍摄时间验证。
exifread库，如果选择最新的3.0.0版本，则处理HEIC文件报错。pip install exifread<3，选择小于3.0.0版本即可。

HEIC文件和对应JPG文件，全部通过了taken time的check。
全部的JPG - JPG文件，没有taken time，file size也不相同，决定进行保留。经检查，是JPG文件，经过了图像处理，得到另一个JPG文件。所以，全保留了。

In [16]:
for idx, fname in enumerate(delfile):
    if Path(fname).suffix == '.JPG':
        fh = open(fname, 'rb')
        tags = exifread.process_file(fh, details=False)
        if 'EXIF DateTimeOriginal' in tags:
            orig_date_del = tags['EXIF DateTimeOriginal']
            fh = open(reserve_file[idx], 'rb')
            tags2 = exifread.process_file(fh, details=False)
            if 'EXIF DateTimeOriginal' in tags2:
                orig_date_res = tags2['EXIF DateTimeOriginal']
                if str(orig_date_res) != str(orig_date_del):
                    print(fname + ' and ' + reserve_file[idx] + 'taken date not match')
            else: # 一个有拍摄日期，一个没有
                print('one has taken date, the other no')
        else: # file size 
            if os.path.getsize(fname) != os.path.getsize(reserve_file[idx]):
                print(fname + ' ' + reserve_file[idx] + ' file size NO match')
                print(os.path.getsize(fname))
                print(os.path.getsize(reserve_file[idx]))
                delfile.pop(idx)
                reserve_file.pop(idx)

                


../iPhoneXR_undocumented/102APPLE/IMG_2325.JPG ../wps_photo/2020-06/IMG_2325.JPG file size NO match
154204
111111
../iPhoneXR_undocumented/102APPLE/IMG_2345.JPG ../wps_photo/2020-06/IMG_2345.JPG file size NO match
90814
61714
../iPhoneXR_undocumented/102APPLE/IMG_2346.JPG ../wps_photo/2020-06/IMG_2346.JPG file size NO match
210090
234800
../iPhoneXR_undocumented/102APPLE/IMG_2350.JPG ../wps_photo/2020-06/IMG_2350.JPG file size NO match
227364
438908
../iPhoneXR_undocumented/102APPLE/IMG_2437.JPG ../wps_photo/2020-07/IMG_2437.JPG file size NO match
697002
356969
../iPhoneXR_undocumented/102APPLE/IMG_2438.JPG ../wps_photo/2020-07/IMG_2438.JPG file size NO match
664911
359046
../iPhoneXR_undocumented/102APPLE/IMG_2439.JPG ../wps_photo/2020-07/IMG_2439.JPG file size NO match
697002
386193
../iPhoneXR_undocumented/102APPLE/IMG_2440.JPG ../wps_photo/2020-07/IMG_2440.JPG file size NO match
664911
351909
../iPhoneXR_undocumented/102APPLE/IMG_2441.JPG ../wps_photo/2020-06/IMG_2441.JPG file size

开始删除文件

In [17]:
for fname in delfile:
    if os.path.exists(fname):
        os.remove(fname)
        print(fname+' removed')

../iPhoneXR_undocumented/100APPLE/IMG_0226.MOV removed
../iPhoneXR_undocumented/100APPLE/IMG_0333.MOV removed
../iPhoneXR_undocumented/100APPLE/IMG_0425.MOV removed
../iPhoneXR_undocumented/100APPLE/IMG_0428.JPG removed
../iPhoneXR_undocumented/100APPLE/IMG_0432.JPG removed
../iPhoneXR_undocumented/100APPLE/IMG_0433.JPG removed
../iPhoneXR_undocumented/100APPLE/IMG_0434.JPG removed
../iPhoneXR_undocumented/100APPLE/IMG_0436.JPG removed
../iPhoneXR_undocumented/100APPLE/IMG_0437.MOV removed
../iPhoneXR_undocumented/100APPLE/IMG_0439.JPG removed
../iPhoneXR_undocumented/100APPLE/IMG_0441.MOV removed
../iPhoneXR_undocumented/100APPLE/IMG_0442.MOV removed
../iPhoneXR_undocumented/100APPLE/IMG_0443.JPG removed
../iPhoneXR_undocumented/100APPLE/IMG_0444.JPG removed
../iPhoneXR_undocumented/100APPLE/IMG_0445.JPG removed
../iPhoneXR_undocumented/100APPLE/IMG_0446.JPG removed
../iPhoneXR_undocumented/100APPLE/IMG_0447.JPG removed
../iPhoneXR_undocumented/100APPLE/IMG_0448.JPG removed
../iPhoneX

In [14]:
# for k,v in tags.items():
#     print(k + ' value: ',v)
os.path.getsize(fname)

str(tags['EXIF ExifImageWidth'])
if 'EXIF DateTimeOriginal' in tags:
    print('exist')
tags['EXIF DateTimeOriginal']

str(orig_date_del)


exist


'2021:04:20 11:40:53'